In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

##  Client-Server Environment Simulator

In [ ]:
class Server:
    def __init__(self, capacity):
        self.capacity = capacity  # Requests per second
        self.active = False  # Whether the server is active

In [ ]:
class ClientServerEnv:
    def __init__(self, max_servers=10, demand_pattern='poisson'):
        # Server settings
        self.max_servers = max_servers
        self.servers = [Server(capacity=10) for _ in range(max_servers)]
        self.active_servers = 1  # Start with 1 server
        
        # Client demand
        self.demand_pattern = demand_pattern  # 'poisson' or 'sinusoidal'
        self.queue = []
        
        # RL settings
        self.state_dim = 3  # [active_servers, queue_length, demand]
        self.action_space = [-1, 0, +1]  # Actions: remove, keep, add
        
    def reset(self):
        self.active_servers = 1
        self.queue = []
        return self._get_state()
    
    def _get_state(self):
        # Normalized state: [active_servers, queue_length, demand]
        return np.array([
            self.active_servers / self.max_servers,
            len(self.queue) / 100,  # Assume max queue=100
            self._generate_demand() / 20  # Normalize demand
        ])
    
    def _generate_demand(self):
        if self.demand_pattern == 'poisson':
            return np.random.poisson(lam=10)  # Avg 10 requests/sec
        elif self.demand_pattern == 'sinusoidal':
            return 10 + 10 * np.sin(self.time * 0.1)  # Time-varying
    
    def step(self, action):
        # Update server count (clamp between 1 and max_servers)
        self.active_servers = np.clip(
            self.active_servers + action, 1, self.max_servers
        )
        
        # Generate new requests
        demand = self._generate_demand()
        self.queue.extend([1] * int(demand))
        
        # Process requests
        processed = min(
            len(self.queue),
            self.active_servers * self.servers[0].capacity  # All servers have same capacity
        )
        self.queue = self.queue[processed:]
        
        # Calculate reward
        reward = self._calculate_reward(processed)
        
        # Next state
        next_state = self._get_state()
        done = False
        return next_state, reward, done, {}
    
    def _calculate_reward(self, processed):
        # Penalize under-provisioning (queue length)
        queue_penalty = len(self.queue) * 0.1
        # Penalize over-provisioning (idle servers)
        idle_servers = self.active_servers - (processed / 10)
        idle_penalty = idle_servers * 0.5
        # Reward processed requests
        processed_reward = processed * 1.0
        return processed_reward - queue_penalty - idle_penalty